In [ ]:
#categorize all pixels in the bird's view into left group and right group
#fit left and right quadratic curve lines using detected pixels in left group and right group respectively

import cv2  
import numpy as np
import math


def find_lane(frame):
    y,x=frame.shape
    #print(frame.shape)
    nonzero=np.transpose(np.nonzero(frame))
    L=[m for m in nonzero if m[1]<0.5*x]
    R=[n for n in nonzero if n[1]>0.5*x]    
    #print('L',L)
    #print('R',R)
    return L,R

    
     

def fit_lanes(points_l, points_r, fit_globally=True):
    yl,xl=np.transpose(points_l)
    yr,xr=np.transpose(points_r)
    fit_ret={}
    if fit_globally:
        parameters = np.polyfit(yl, xl, 2)
        fit_ret.update({'al':parameters[0], 'bl':parameters[1],'cl':parameters[2],
                       'ar':parameters[0], 'br':parameters[1],'cr':parameters[2]})
    else:
        parameters_l= np.polyfit(yl, xl, 2)
        parameters_r= np.polyfit(yr, xr, 2)
        fit_ret.update({'al':parameters_l[0], 'bl':parameters_l[1],'cl':parameters_l[2],
                       'ar':parameters_r[0], 'br':parameters_r[1],'cr':parameters_r[2]})

    return fit_ret


def line_fit(binary_warped,fit_globally=True, quadratic=True):
    """
    Find and fit lane lines
    """
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = (np.dstack((binary_warped, binary_warped, binary_warped))*255).astype('uint8')
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[100:midpoint]) + 100
    rightx_base = np.argmax(histogram[midpoint:-100]) + midpoint
    #leftx_base = np.argmax(histogram[437-5:437+5]) + 437-5
    #rightx_base = np.argmax(histogram[800-5: 800+5]) + 800-5
    #leftx_base = 437
    #rightx_base = 800
    x_base_first=np.argmax(histogram[140-20:140+20]) + 140-20
    x_base_second=np.argmax(histogram[310-20:310+20]) + 310-20
    x_base_third=np.argmax(histogram[570-20:570+20]) + 570-20
    x_base_fourth=np.argmax(histogram[900-100:900+100]) + 900-100
    x_base_fifth=np.argmax(histogram[1136-50:1136+50]) + 1136-50

    # Choose the number of sliding windows
    nwindows = 6
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    #leftx_current = leftx_base
    #rightx_current = rightx_base
    x_current_first=x_base_first
    x_current_second=x_base_second
    x_current_third=x_base_third
    x_current_fourth=x_base_fourth
    x_current_fifth=x_base_fifth
    # Set the width of the windows +/- margin
    margin = 70
    # Set minimum number of pixels found to recenter window
    minpix_l = 10
    minpix_r = 10
    # Create empty lists to receive left and right lane pixel indices
    #left_lane_inds = []
    #right_lane_inds = []
    first_lane_inds= []
    second_lane_inds= []
    third_lane_inds= []
    fourth_lane_inds= []
    fifth_lane_inds= []
    

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - (window)*window_height
        #win_xleft_low = leftx_current - margin
        #win_xleft_high = leftx_current + margin
        #win_xright_low = rightx_current - margin
        #win_xright_high = rightx_current + margin
        xfirst_low=x_current_first-margin
        xfirst_high=x_current_first+margin
        xsecond_low=x_current_second-margin
        xsecond_high=x_current_second+margin
        xthird_low=x_current_third-margin
        xthird_high=x_current_third+margin
        xfourth_low=x_current_fourth-margin
        xfourth_high=x_current_fourth+margin
        xfifth_low=x_current_fifth-margin
        xfifth_high=x_current_fifth+margin
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,0,255), 3)
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,0,255), 3)
        cv2.rectangle(out_img,(xfirst_low,win_y_low),(xfirst_high,win_y_high),(0,0,255), 3)
        cv2.rectangle(out_img,(xsecond_low,win_y_low),(xsecond_high,win_y_high),(0,0,255), 3)
        cv2.rectangle(out_img,(xthird_low,win_y_low),(xthird_high,win_y_high),(0,0,255), 3)
        cv2.rectangle(out_img,(xfourth_low,win_y_low),(xfourth_high,win_y_high),(0,0,255), 3)
        cv2.rectangle(out_img,(xfifth_low,win_y_low),(xfifth_high,win_y_high),(0,0,255), 3)
        # Identify the nonzero pixels in x and y within the window
        #good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        #good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        good_first_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= xfirst_low) & (nonzerox < xfirst_high)).nonzero()[0]
        good_second_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= xsecond_low) & (nonzerox < xsecond_high)).nonzero()[0]
        good_third_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= xthird_low) & (nonzerox < xthird_high)).nonzero()[0]
        good_fourth_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= xfourth_low) & (nonzerox < xfourth_high)).nonzero()[0]
        good_fifth_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= xfifth_low) & (nonzerox < xfifth_high)).nonzero()[0]
        # Append these indices to the lists
        #left_lane_inds.append(good_left_inds)
        #right_lane_inds.append(good_right_inds)
        first_lane_inds.append(good_first_inds)
        second_lane_inds.append(good_second_inds)
        third_lane_inds.append(good_third_inds)
        fourth_lane_inds.append(good_fourth_inds)
        fifth_lane_inds.append(good_fifth_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        #if len(good_left_inds) > minpix_l:
            #leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        #if len(good_right_inds) > minpix_r:
            #rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        if len(good_first_inds) > minpix_r:
            x_current_first = np.int(np.mean(nonzerox[good_first_inds]))
        if len(good_second_inds) > minpix_r:
            x_current_second = np.int(np.mean(nonzerox[good_second_inds]))
        if len(good_third_inds) > minpix_r:
            x_current_third = np.int(np.mean(nonzerox[good_third_inds]))
        if len(good_fourth_inds) > minpix_r:
            x_current_fourth = np.int(np.mean(nonzerox[good_fourth_inds]))
        if len(good_fifth_inds) > minpix_r:
            x_current_fifth = np.int(np.mean(nonzerox[good_fifth_inds]))

    # Concatenate the arrays of indices
    #left_lane_inds = np.concatenate(left_lane_inds)
    #right_lane_inds = np.concatenate(right_lane_inds)
    first_lane_inds = np.concatenate(first_lane_inds)
    second_lane_inds = np.concatenate(second_lane_inds)
    third_lane_inds = np.concatenate(third_lane_inds)
    fourth_lane_inds = np.concatenate(fourth_lane_inds)
    fifth_lane_inds = np.concatenate(fifth_lane_inds)

    # Extract left and right line pixel positions
    #leftx = nonzerox[left_lane_inds]
    #lefty = nonzeroy[left_lane_inds]
    #rightx = nonzerox[right_lane_inds]
    #righty = nonzeroy[right_lane_inds]
    firstx = nonzerox[first_lane_inds]
    firsty = nonzeroy[first_lane_inds]
    secondx = nonzerox[second_lane_inds]
    secondy = nonzeroy[second_lane_inds]
    thirdx = nonzerox[third_lane_inds]
    thirdy = nonzeroy[third_lane_inds]
    fourthx = nonzerox[fourth_lane_inds]
    fourthy = nonzeroy[fourth_lane_inds]
    fifthx = nonzerox[fifth_lane_inds]
    fifthy = nonzeroy[fifth_lane_inds]

    # Fit a second order polynomial to each
    fit_ret={}
    if fit_globally:
        if quadratic:
            parameters_1 = np.polyfit(firsty, firstx, 2)
            parameters_2 = np.polyfit(secondy, secondx, 2)
            parameters_3 = np.polyfit(thirdy, thirdx, 2)
            parameters_4 = np.polyfit(fourthy, fourthx, 2)
            parameters_5 = np.polyfit(fifthy, fifthx, 2)
            fit_ret.update({'a1':parameters_1[0], 'b1':parameters_1[1],'c1':parameters_1[2],
                            'a2':parameters_1[0], 'b2':parameters_1[1],'c2':parameters_2[2],
                            'a3':parameters_1[0], 'b3':parameters_1[1],'c3':parameters_3[2],
                            'a4':parameters_1[0], 'b4':parameters_1[1],'c4':parameters_4[2],
                            'a5':parameters_1[0], 'b5':parameters_1[1],'c5':parameters_5[2]})
        else:
            parameters_1 = np.polyfit(firsty, firstx, 3)
            parameters_2 = np.polyfit(secondy, secondx, 3)
            parameters_3 = np.polyfit(thirdy, thirdx, 3)
            parameters_4 = np.polyfit(fourthy, fourthx, 3)
            parameters_5 = np.polyfit(fifthy, fifthx, 3)
            fit_ret.update({'a1':parameters_1[0], 'b1':parameters_1[1],'c1':parameters_1[2],'d1':parameters_1[3],
                            'a2':parameters_1[0], 'b2':parameters_1[1],'c2':parameters_1[2],'d2':parameters_2[3],
                            'a3':parameters_1[0], 'b3':parameters_1[1],'c3':parameters_1[2],'d3':parameters_3[3],
                            'a4':parameters_1[0], 'b4':parameters_1[1],'c4':parameters_1[2],'d4':parameters_4[3],
                            'a5':parameters_1[0], 'b5':parameters_1[1],'c5':parameters_1[2],'d5':parameters_5[3]})
    else:
        if quadratic:
            parameters_1 = np.polyfit(firsty, firstx, 2)
            parameters_2 = np.polyfit(secondy, secondx, 2)
            parameters_3 = np.polyfit(thirdy, thirdx, 2)
            parameters_4 = np.polyfit(fourthy, fourthx, 2)
            parameters_5 = np.polyfit(fifthy, fifthx, 2)
            fit_ret.update({'a1':parameters_1[0], 'b1':parameters_1[1],'c1':parameters_1[2],
                            'a2':parameters_2[0], 'b2':parameters_2[1],'c2':parameters_2[2],
                            'a3':parameters_3[0], 'b3':parameters_3[1],'c3':parameters_3[2],
                            'a4':parameters_4[0], 'b4':parameters_4[1],'c4':parameters_4[2],
                            'a5':parameters_5[0], 'b5':parameters_5[1],'c5':parameters_5[2]})
        else:
            parameters_1 = np.polyfit(firsty, firstx, 3)
            parameters_2 = np.polyfit(secondy, secondx, 3)
            parameters_3 = np.polyfit(thirdy, thirdx, 3)
            parameters_4 = np.polyfit(fourthy, fourthx, 3)
            parameters_5 = np.polyfit(fifthy, fifthx, 3)
            fit_ret.update({'a1':parameters_1[0], 'b1':parameters_1[1],'c1':parameters_1[2],'d1':parameters_1[3],
                            'a2':parameters_2[0], 'b2':parameters_2[1],'c2':parameters_2[2],'d2':parameters_2[3],
                            'a3':parameters_3[0], 'b3':parameters_3[1],'c3':parameters_3[2],'d3':parameters_3[3],
                            'a4':parameters_4[0], 'b4':parameters_4[1],'c4':parameters_4[2],'d4':parameters_4[3],
                            'a5':parameters_5[0], 'b5':parameters_5[1],'c5':parameters_5[2],'d5':parameters_5[3]})
    return fit_ret, out_img





def draw_lanes(frame,parameters,quadratic=True):
    #pts_l=[]
    #pts_r=[]
    pts_1=[]
    pts_2=[]
    pts_3=[]
    pts_4=[]
    pts_5=[]
    y=np.linspace(0,frame.shape[0]-1,num=frame.shape[0])
    if quadratic:
        #xl=parameters['al']*y**2+parameters['bl']*y+parameters['cl']
        #xr=parameters['ar']*y**2+parameters['br']*y+parameters['cr']
        x1=parameters['a1']*y**2+parameters['b1']*y+parameters['c1']
        x2=parameters['a2']*y**2+parameters['b2']*y+parameters['c2']
        x3=parameters['a3']*y**2+parameters['b3']*y+parameters['c3']
        x4=parameters['a4']*y**2+parameters['b4']*y+parameters['c4']
        x5=parameters['a5']*y**2+parameters['b5']*y+parameters['c5']
    else:
        #xl=parameters['al']*y**3+parameters['bl']*y**2+parameters['cl']*y+parameters['dl']
        #xr=parameters['ar']*y**3+parameters['br']*y**2+parameters['cr']*y+parameters['dr']
        x1=parameters['a1']*y**3+parameters['b1']*y**2+parameters['c1']*y+parameters['d1']
        x2=parameters['a2']*y**3+parameters['b2']*y**2+parameters['c2']*y+parameters['d2']
        x3=parameters['a3']*y**3+parameters['b3']*y**2+parameters['c3']*y+parameters['d3']
        x4=parameters['a4']*y**3+parameters['b4']*y**2+parameters['c4']*y+parameters['d4']
        x5=parameters['a5']*y**3+parameters['b5']*y**2+parameters['c5']*y+parameters['d5']
    
    #pts_l=np.array([np.transpose(np.vstack([xl,y]))])
    #pts_r=np.array([np.flipud(np.transpose(np.vstack([xr,y])))])
    pts_1=np.array([np.transpose(np.vstack([x1,y]))])
    pts_2=np.array([np.flipud(np.transpose(np.vstack([x2,y])))])
    polygon = np.zeros_like(frame)
    pts = np.hstack((pts_1, pts_2))
    cv2.fillPoly(polygon, np.int_([pts]), (0,0,200))
    
    pts_2=np.array([np.transpose(np.vstack([x2,y]))])
    pts_3=np.array([np.flipud(np.transpose(np.vstack([x3,y])))])
    pts = np.hstack((pts_2, pts_3))
    cv2.fillPoly(polygon, np.int_([pts]), (200,0,0))
    
    pts_3=np.array([np.transpose(np.vstack([x3,y]))])
    pts_4=np.array([np.flipud(np.transpose(np.vstack([x4,y])))])
    pts = np.hstack((pts_3, pts_4))
    cv2.fillPoly(polygon, np.int_([pts]), (0,200,0))
    
    pts_4=np.array([np.transpose(np.vstack([x4,y]))])
    pts_5=np.array([np.flipud(np.transpose(np.vstack([x5,y])))])
    pts = np.hstack((pts_4, pts_5))
    cv2.fillPoly(polygon, np.int_([pts]), (0,200,200))
    
    return polygon


#run the test video
cap = cv2.VideoCapture('project_video.mp4',0)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('multi-lanes detection on highway.avi',fourcc, 20.0, (1280,720))
frame_count = 1

while(cap.isOpened()):
#while True:  
    _,frame = cap.read()  
    #print('frame',frame.shape)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(3,3),0)
    edges_white=cv2.Canny(gray,150,200)
    
    HSL = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
    lower = np.uint8([ 10,   0,  100])
    upper = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(HSL, lower, upper)
    YellowImg = cv2.bitwise_and(frame, frame, mask=yellow_mask)
    GrayImg = cv2.cvtColor(YellowImg, cv2.COLOR_BGR2GRAY)
    blurredImg = cv2.GaussianBlur(GrayImg, (3, 3), 0)
    edges_yellow = cv2.Canny(blurredImg, threshold1=100, threshold2=210)
    
    edges=cv2.addWeighted(edges_white, 1.0, edges_yellow, 1.0,0)
    
    
    original=frame.copy()
    n1,n2,n3=original.shape
    
    # implement homography transformation
    #src = np.float32([[237, 720],[1100, 720],[440, 450],[840, 450]])
    #dst = np.float32([[437, 720],[800, 720],[300, 0],[980, 0]])
    #src = np.float32([[237, 720],[1100, 720],[540, 450],[740, 450]])
    #dst = np.float32([[277, 720],[540, 720],[140, 0],[765, 0]])
    src = np.float32([[237, 720],[1100, 720],[540, 450],[740, 450]])
    dst = np.float32([[297, 720],[520, 720],[160, 0],[745, 0]])
    #src = np.float32([[200, 720],[1100, 720],[590, 450],[685, 450]])
    #dst = np.float32([[150, 720],[1020, 720],[300, 0],[980, 0]])
    world=np.array(src)
    ground=np.array(dst)
    h,status=cv2.findHomography(world,ground,cv2.RANSAC,5,maxIters=100, confidence=0.99)
    h_inv,status=cv2.findHomography(world,ground,cv2.RANSAC,5,maxIters=100, confidence=0.99)#h_inv for unwarpping the bird's view 
    projected_transformation=cv2.warpPerspective(original,h,(n2,n1),flags=cv2.INTER_LINEAR)
    projected_edges=cv2.warpPerspective(edges,h,(n2,n1),flags=cv2.INTER_LINEAR)
    

    
   
    parameters=line_fit(projected_edges,False,True)[0]
    algorithm=line_fit(projected_edges,False,True)[1]
    #points=find_lane(projected_edges)                                #find lanes in bird's view
    #parameters=fit_lanes(points[0], points[1], fit_globally=False)   #fit the quadratic curve model and get curve parameters
    #print('parameters=',parameters)
    polygon=draw_lanes(original,parameters,True)                          #draw curve lines using the calculated parameters
    
    
    
    #projected back to the world view
    h,status=cv2.findHomography(ground,world,cv2.RANSAC,5,maxIters=100, confidence=0.99)
    h_inv,status=cv2.findHomography(ground,world,cv2.RANSAC,5,maxIters=100, confidence=0.99)#h_inv for unwarpping the bird's view 
    bird_view=cv2.addWeighted(projected_transformation, 1.0, polygon, 0.5,0)
    polygon_back=cv2.warpPerspective(polygon,h,(n2,n1),flags=cv2.INTER_LINEAR)
    outcome=cv2.addWeighted(original, 1.0, polygon_back, 0.5,0)# add the polygon layer to the oringinal frame
    
    
    #cv2.imshow("canny",edges)
    cv2.imshow('projected edge', projected_edges)
    #cv2.imshow('projected transformation',projected_transformation)
    #cv2.imshow('polygon',polygon)
    cv2.imshow('bird view',bird_view)
    cv2.imshow('outcome',outcome)
    cv2.imshow('algorithm',algorithm)

    
    #out.write(outcome)
    
    #params = []  
    #params.append(cv.CV_IMWRITE_PXM_BINARY)  
    #params.append(1)  
    #cv2.imwrite("world" + "_%d.jpg" % frame_count, out, params)
    #cv2.imwrite('algorithm' + "_%d.jpg" % frame_count, algorithm, params)
    #cv2.imwrite('bird view' + "_%d.jpg" % frame_count, bird_view, params)
    #cv2.imwrite('outcome' + "_%d.jpg" % frame_count, outcome, params)
    #frame_count = frame_count + 1  
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cap.release()
out.release()
cv2.destroyAllWindows()